In [1]:
#!/usr/bin/env python
%load_ext autoreload
%autoreload 2

from utils.preprocess import normalize
from utils.parser import nlp
from utils.auto_suggest import auto_suggest
from utils.grammar import iterate_all_patterns

In [ ]:
def diff_after_before(text):
    before, after , count = [], [] , 0
    for token in text.split(' '):
        if token.startswith('{+'):
            after_word = token[2:-2].split('//')[0]
            after.append(after_word)
        elif token.startswith('[-'):
            if token.endswith('+}'):
                delete_tmp, insert_tmp = token[2:-2].split('-]{+')
                delete = delete_tmp.split('//')[0]
                insert = insert_tmp.split('//')[0]
                before.append(delete)
                after.append(insert)
            else:
                before_word = token[2:-2].split('//')[0]
                before.append(before_word)
        else:
            before.append(token)
            after.append(token)
    return ' '.join(after).replace('\u3000',' ')

In [ ]:
test = []
for line in open('/atom/corpus/written/EFCAM/EFCAM.error.txt', 'r', encoding='utf8'):
    parse = nlp(diff_after_before(line))
    for i, tk in enumerate(parse):
        if tk.tag_.startswith('JJ'):
            test.append(' '.join([tk.text for tk in parse][:i+1]))
            break
            
    if len(test) > 400:
        break

In [14]:
tests = '''It has integrated functional
The bread is much larger
Now there are only 3 x 2 bedroom ones available
John encountered a strange
As he was approaching the corner at Elm and Fourth, he saw a young
Tim prepared to stop them after they vandalized the local
But suddenly, the young
Finally, they found the young
Now the couple has still not been caught which makes Leo scared
Firstly, you should only bring little
Hit the ball as many
To score a point, try to make the opposite
If you ask me what you want to do in the following
I would like to fluently use a foreign
Then, in my company, most
I think administration is better
The person in that position must work hard and is responsible
When I think of the journey, I feel it was very happy
I was surprised
Though we only stayed in USA for two days, it still left a deep
They come to a small
They do not have enough
Isabella and John are tired
I am willing
Grandfather is an old
You are addicted
It is my great
It would be very appreciated if you could spare a few
First of all, I would be a very good
I promise I will take every single
I promise we will have different
I will convince our principal to get more
I like music and digital
Later, we moved into the city to continue with my high
When I graduated from university, I found my first
I water the plants in the morning every day and then I take the dogs for a walk if it is not rainy
So studying English is the first
I need these extra
The viewing figures show that unemployed
You should make them most interesting
You must be more professional
It describes life where the singer lived before and his current
Although the lyrics are not happy
This is a very fashionable
When you wear them, you will detect yourself higher
Although you walk for a long
It is advisable
You have been late
The lunch was unbelievably fantastic
I was full
'If you allow me to give some suggestions, I think the music is not suitable
I am looking for a challenging
He has short
If we do not have many
If it is rainy
Please consider how much
Perhaps, you could bring home-made
Also, you could cook dinner at home or try a cheap
The most important
She wears long red shorts and a white
She has long
He is cool
He has fairly short
Mary and Jim are my Asian
I hope I could find a lover who would also love me deeply to establish a warmhearted
Dear Julia, I a, very angry
It gives me great
First, I will get a job in a chemical
Due to my promotion, I feel that my current
Today, the weather is rainy
It is going to be sunny
I could try to do special
At that time, my pretty
 Hi Anna, I have found an absolutely amazing
This job provides very good
I urge you to apply as soon as possible
Looking forward to your prompt
I am intelligent
Let me tell you why you should apply this exciting
I believe it is much higher
Furthermore, just as you expected, the benefits include full
I believe this is a good
The player with the most
They are cheaper
I have a small
I love every day , because it is a new
First , I will work hard for my recent
There is a tub filled with water and an empty
There is an empty
The member uses the bottle to transport the water to the empty
I work in a primary
I always work with children and it is very interesting
I work in a international
My job is too difficult
This afternoon is going to rain, so I will go to the museum and eat at one of the great
The Iyrics are beautiful
At that moment, I heard the loud
I called to police with a public
It is too far from a cosy
The captain was unfriendly
There is a good
After I finished high
I work for long
I am waiting for a long
Thousands of people are celebrating it together, and they will wear beautiful
It is going to be amazing
Maybe the clothes and shoes are sporting and comfortable
It is going to be great
He was a dangerous
Some of the valuable
All employees are required to wear natural'''.split('\n')

In [12]:
test = 'Then, in my company, most'
parse = nlp(test)
sugs = auto_suggest(parse, iterate_all_patterns(parse))

In [15]:
results = []
for test in tests:        
    parse = nlp(test)
    sugs = auto_suggest(parse, iterate_all_patterns(parse))
    sugs = filter(lambda sug: not sug['level'].startswith('A'), sugs['patterns'])
    ngrams = [sug['ngram'] + ' (' + sug['pattern'] + ')' for sug in sugs]

    if len(ngrams) < 3: continue 
    results.append((test, ngrams[:3]))
    
    
ws = open('testdata.txt', 'w', encoding='utf8')

start = '''var form = FormApp.create('English Level-Up');\n\n'''

template = ''

for i, (line, ngrams) in enumerate(results[:50]):
    print("="*20, file=ws)
    print(line, file=ws)
    print(file=ws)
    print('\n'.join(ngrams), file=ws)
    print('='*20, file=ws)
    print(file=ws)
    
    if i % 10 == 0:
        template += "form.addPageBreakItem().setTitle(" + str(i//10 + 1) + ");\n\n"
    options = ','.join(["'" + ngram + "'" for ngram in ngrams[:3]]) + ",'NONE'"
    template += "form.addCheckboxItem().setTitle('" + line + "').setChoiceValues([" + options + "]).showOtherOption(false).setRequired(true);\n\n"

print(start + template)
ws.close()

['Please', 'consider', 'how', 'much']
32 3 5
['Please', 'consider', 'how', 'much']
32 3 5
['Please', 'consider', 'how', 'much']
115 3 5
['Please', 'consider', 'how', 'much']
115 3 5
var form = FormApp.create('English Level-Up');

form.addPageBreakItem().setTitle(1);

form.addCheckboxItem().setTitle('It has integrated functional').setChoiceValues(['as functional as (as adj. as / so adj. that)','functional items such as (phrase-adj.)','purely functional terms (adv./adj. adj. n.)','NONE']).showOtherOption(false).setRequired(true);

form.addCheckboxItem().setTitle('Now there are only 3 x 2 bedroom ones available').setChoiceValues(['data available to (phrase-adj.)','make themselves available (make n. adj.)','publicly available data (adv./adj. adj. n.)','NONE']).showOtherOption(false).setRequired(true);

form.addCheckboxItem().setTitle('John encountered a strange').setChoiceValues(['very strange feeling (adv./adj. adj. n.)','strange enough (adj. enough)','nothing strange (pronoun adj.)','NON